In [2]:
!pip install Faker
!pip install holidays

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
from faker import Faker
import pandas as pd
import datetime
import holidays
import random
import datetime
import holidays

# Data generation

Generated data based on our best approximations.

In [4]:
def generate_work_schedule(birthdate, year):
    
    dates = [datetime.date(year, 1, 1) + datetime.timedelta(days=i) for i in range(365)]
    hungary_holidays = holidays.Hungary(years=year)
    workdays = [d for d in dates if d.weekday() < 5 and d not in hungary_holidays]
    num_vacation_days = int(len(workdays) * 0.12)
    vacation_days = []

    if birthdate in workdays and random.random() < 0.9:
        workdays.remove(birthdate)
        vacation_days.append(birthdate)
    
    while num_vacation_days > 0:
        try:
            vacation_day = int(np.random.normal(180, 30)) if num_vacation_days > 9 else faker.random_int(min=1, max=len(dates)-8)
            while dates[vacation_day] not in workdays:
                vacation_day = int(np.random.normal(180, 30))
            lenght = faker.random_int(min=1, max=7)
            num_vacation_days -= lenght
            while lenght > 0:
                if dates[vacation_day] in workdays:
                    workdays.remove(dates[vacation_day])
                    vacation_days.append(dates[vacation_day])
                    lenght -= 1
                vacation_day += 1
        except IndexError:
            continue


    home_office_days = []
    home_office_schedule = [False] * 5
    home_office_habits = []
    size = np.random.randint(1, 3)
    for i in range(size):
        home_office_habits.append(np.random.randint(1, 5))
    for i in home_office_habits:
        home_office_schedule[i] = True

    for day in workdays:
        rnd = np.random.random() < 0.9
        if (home_office_schedule[day.weekday()] and rnd) or not rnd:
            home_office_days.append(day)

    return {'vacation_days': vacation_days, 'home_office_days': home_office_days}

In [5]:
faker = Faker()
data = []
hu_holidays = holidays.HU()


for i in range(2000):
    row = {}
    row['name'] = faker.name()
    row['email'] = faker.email()
    row['gender'] = np.random.random() > 0.6
    row['hired_ago'] = np.random.normal(6*365, 500)
    advance = np.random.normal(3*365, 180)
    row['last_advance_ago'] = advance if advance < row['hired_ago'] else row['hired_ago']
    row['birth_ago'] = np.random.normal(40*365, 8*365)
    row['salary'] = np.random.normal(700000, 100000)
    row['position'] = int(np.random.normal(3, 0.5))
    row['has_ps'] = row['position'] > 3 if np.random.random() < 0.8 else row['position'] <= 3
    row['arrival_mean'] = np.random.normal(8, 1)
    row['arrival_stdv'] = np.random.normal(1, 0.2)
    row['depart_mean'] = np.random.normal(8 + np.random.normal(8, 0.5), 1)
    row['depart_stdv'] = np.random.normal(2, 0.2)
    row['license_plate_cnt'] = 1 if np.random.random() > 0.8 else 2 if np.random.random() > 0.9 else 3
    row['work_area'] = faker.random_int(min=1, max=3) 
    row['reservation_ratio'] = np.random.normal(0.8, 0.1) #el van tárolva összes reservation/elment
    row['giveup_ratio'] = np.random.normal(1, 0.2) #hányszor adta fel a helyét/hányszor nem ment el
    off_days = generate_work_schedule(datetime.datetime(1973, 2, 24, 12, 0, 0), 2022)
    vacation_dates = off_days['vacation_days']
    home_office_dates = off_days['home_office_days']
    dates = [datetime.date(2022, 1, 1) + datetime.timedelta(days=i) for i in range(365)]
    for i in range(len(dates)):
      if dates[i] in vacation_dates:
        row[str(i) ] = "V"
      else:
         if dates[i] in home_office_dates:
          row[str(i)] = "H"
         else:
          row[str(i)] = "W"
    data.append(row)
    

df = pd.DataFrame(data)

# faszi beszéde valahány 
#ha meg akarod érteni hogy mi fog történni holnap akkor először meg kell értened, hogy mi fog törénni holnapután
#i turns out we can learn so much more of people's parking habits than a.


# Exporting/Importing data


In [7]:
#exporting data for app integration
df.to_csv('output.csv')

In [ ]:
#importing data for training
df = pd.read_csv('output.csv')

In [9]:
import os
os.getcwd()

'/content'

# Analyzation, validation

Analyze and validate generated date whith basic statistical tools.

In [5]:
df.describe()

,hired_ago,last_advance_ago,birth_ago,salary,position,arrival_mean,arrival_stdv,depart_mean,depart_stdv,license_plate_cnt,work_area,reservation_ratio,giveup_ratio
count,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,2187.543751,1092.905909,14650.681530,6.999739e+05,2.500000,8.004209,0.998919,15.993654,1.992544,2.534500,1.997500,0.800672,0.996484
std,495.186459,177.621265,2857.415991,1.013188e+05,0.577206,1.004834,0.198799,1.124284,0.198450,0.786205,0.815982,0.100529,0.201220
min,246.271445,246.271445,4323.768841,3.624590e+05,1.000000,4.540304,0.274275,12.284088,1.379179,1.000000,1.000000,0.451715,0.252104
25%,1855.399331,975.230923,12739.823190,6.322217e+05,2.000000,7.341683,0.865759,15.227348,1.864078,2.000000,1.000000,0.728004,0.857922
50%,2185.980608,1096.449287,14674.264890,6.994773e+05,2.000000,7.975842,1.005420,16.007157,1.988904,3.000000,2.000000,0.800155,0.997834
75%,2519.930840,1207.404446,16535.976666,7.708996e+05,3.000000,8.658206,1.130359,16.757612,2.124931,3.000000,3.000000,0.870294,1.132647
max,4084.759252,1771.468804,26033.352330,1.050385e+06,4.000000,11.366477,1.700334,20.648658,2.643621,3.000000,3.000000,1.164787,1.703285


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 382 entries, name to 364
dtypes: bool(2), float64(10), int64(3), object(367)
memory usage: 5.8+ MB


In [7]:
print(df)

                      name                      email  gender    hired_ago  \
0     Dr. Raymond Martinez       angela06@example.com    True  1784.881094   
1             Tony Wallace       teresa97@example.org    True  2272.279126   
2            Rodney Butler   achristensen@example.org   False  2173.320620   
3          Amanda Campbell        david95@example.net    True  1717.751589   
4           Samuel Brennan    stephanie66@example.com    True  2842.991146   
...                    ...                        ...     ...          ...   
1995         Jeffrey Logan  brookswilliam@example.com   False  2376.477987   
1996            Katie Lamb        xhowell@example.net    True  2205.338572   
1997           Mary Carter        casey01@example.org   False  2818.635406   
1998        Katrina Mosley   allenmelissa@example.net   False  2599.062942   
1999          Maria Medina        ygraham@example.net    True  2132.968015   

      last_advance_ago     birth_ago         salary  position  

# Data preparation

Transforming data for the desired format classification model.


In [8]:
def generate_feature_label(df):
    data = []
    for index, row in df.iterrows():
        for i in range(365-4):
            new_row = {}
            new_row['gender'] = row['gender']
            new_row['hired_ago'] = row['hired_ago']
            new_row['last_advance_ago'] = row['last_advance_ago']
            new_row['birth_ago'] = row['birth_ago']
            new_row['salary'] = row['salary']
            new_row['position'] = row['position']
            new_row['has_ps'] = row['has_ps']
            new_row['arrival_mean'] = row['arrival_mean']
            new_row['arrival_stdv'] = row['arrival_stdv']
            new_row['depart_mean'] = row['depart_mean']
            new_row['depart_stdv'] = row['depart_stdv']
            new_row['license_plate_cnt'] = row['license_plate_cnt']
            new_row['work_area'] = row['work_area']
            new_row['reservation_ratio'] = row['reservation_ratio']
            new_row['giveup_ratio'] = row['giveup_ratio']
            for j in range(5):
                new_row[str(j)] = row[str(i+j)]
            data.append(new_row)
    return pd.DataFrame(data)
        

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

data = generate_feature_label(df)
X = data.drop('4', axis=1)
y = data['4']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
cat_cols = [col for col in X.columns if X[col].dtype == 'object']
X = pd.get_dummies(X, columns=cat_cols)


scaler = RobustScaler()
X = pd.DataFrame(scaler.fit_transform(X))

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3)

# Training the model

Setting the model hyperparameters for training, then measuring prediction accuracy on validation set.

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score



model = XGBClassifier(
    learning_rate= 0.8, 
    n_estimators= 10,
    max_depth= 15,
    min_child_weight= 5,
    gamma= 0.1,
    colsample_bytree= 1,
    early_stopping_rounds=15)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=1, eval_metric='mlogloss')

pred = model.predictproba(X_valid)
acc = accuracy_score(y_valid, pred)
print('Accuracy on validation set: ' + str(acc))

[0]	validation_0-mlogloss:0.691249	validation_1-mlogloss:0.703798
[1]	validation_0-mlogloss:0.585141	validation_1-mlogloss:0.608053
[2]	validation_0-mlogloss:0.541995	validation_1-mlogloss:0.574554
[3]	validation_0-mlogloss:0.521847	validation_1-mlogloss:0.561437
[4]	validation_0-mlogloss:0.507954	validation_1-mlogloss:0.553896
[5]	validation_0-mlogloss:0.496208	validation_1-mlogloss:0.547763
[6]	validation_0-mlogloss:0.48698	validation_1-mlogloss:0.543841
[7]	validation_0-mlogloss:0.48164	validation_1-mlogloss:0.543581
[8]	validation_0-mlogloss:0.476738	validation_1-mlogloss:0.543236
[9]	validation_0-mlogloss:0.471217	validation_1-mlogloss:0.543028
Accuracy on validation set: 0.7832963988919668


In [9]:
#from sklearn.model_selection import RandomizedSearchCV

#param_grid = {'n_estimators': [100, 250, 500],
#   "learning_rate": [0.5, 0.7, 0.9],
#   "max_depth": [ 3, 8, 12],
#   "min_child_weight": [3,  5,  7],
#   "gamma": [ 0.2,  0.4,  0.6 ],
#   "colsample_bytree": [  0.4, 0.6, 0.7 ]
#}

#RS_XGB = RandomizedSearchCV(estimator=XGBClassifier(early_stopping_rounds=25), param_distributions=param_grid, n_iter=3, cv=6 ,n_jobs=-1, random_state=0)
#RS_XGB.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)],verbose=0)
#RS_XGB.best_params_

# Exporting the model


In [ ]:
model.save_model('model.bin')